<h1 align="center"> Team Conda</h1> 
<h3 align="center"> Reto </h3> 
<h4 align="center">  Cristian Lazo : mecatronico.lazo@gmail.com</h4> 
<h4 align="center"> Juan Acostupa: juan.acostupa.d@uni.pe   </h4> 
<h4 align="center"> John Taco: john.taco93@gmail.com   </h4> 



--------------------------------

**Resumen  :**  De un conjunto de ubicaciones en un mapa y las distancias respectivas entre ellas, se desea encontrar la distancia mas corta que pueda recorrer todos los puntos.


--------------------------

<a id="1"></a> 

# INDICE


* [INDICE](#1)
* [DATASET](#2)
* [ALGORITMO](#3)
* [IMPORTANDO LIBRERIAS](#4)
* [CARGAR DATA](#5)
* [GENERACIÓN DEL GRAFO](#6)
* [PRUEBO EL ALGORITMO](#7)




<a id="2"></a> 

# DATASET

Consta de 1 archivo txt de las posibles rutas

- AlphaCentauri to Snowdin = 66
- AlphaCentauri to Tambi = 28
- AlphaCentauri to Faerun = 60
- Snowdin to Tambi = 22
- Snowdin to Faerun = 12
- ...

<a id="3"></a> 

# ALGORITMO

Analizando el problema se puede reconocer que es una aplicación de Minimum Spanning Tree y entonces tendríamos varias opciones para hacer el desarrollo:

Un Spanning Tree es un subconjunto de un grafo, que tiene todos los vértices cubiertos con el mínimo número posible de aristas.

Un Minimum Spanning Tree es un árbol de expansión que tiene un peso mínimo que todos los demás árboles de expansión del mismo gráfico.

Los dos algoritmos más populares para el árbol de expansión mínimo son:

 - Algoritmo de Kruskal : 
 
     Se puede mostrar que el algoritmo de Kruskal se ejecuta en tiempo: \begin{equation}\ O(E log E)\end{equation}
     o de manera equivalentE en tiempo:\begin{equation}\ O(E log V)\end{equation}
     donde E es el número de aristas en el gráfico y V es el número de vértices, todo con Estructuras de datos simples.
 - Algoritmo de Prim: 
 
     La complejidad del tiempo es \begin{equation}\ O(VlogV + ElogV) = O (ElogV)\end{equation}, lo que hace que sea el mismo que el algoritmo de Kruskal. Sin embargo, el algoritmo de Prim puede mejorarse utilizando Fibonacci Heaps a \begin{equation}\ O(E + logV)\end{equation}.
 
 Como se puede apreciar, si:  \begin{equation}\ E = V^{2}\end{equation} conviene utilizar Prim. 
 
 **Conclusión** : De acuerdo con la data que muestra un grafo totalmente conectado se plantea desarrollar el algoritmo de Prim para resolver el problema por ser mas eficiente


In [1]:
from PIL import Image
i=Image.open('spannig_tree.jpg')
i = i.resize((300,400))
display(i)

FileNotFoundError: [Errno 2] No such file or directory: 'spannig_tree.jpg'

En la gráfica se puede apreciar que de un grafo se encuentra el arbol de expansión mínima para tener los nodos conectados y con menor distancia.

<a id="4"></a> 

# IMPORTAR LIBRERIAS

In [2]:
import numpy as np
import operator
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

<a id="5"></a> 

# CARGAR DATA

In [3]:
path='./'
target=pd.read_csv(path+'rutas.txt',sep= ' =',header=None)

## Divido los datos en columnas

In [4]:
# Columna inicio=partida del camino
inicio=target[0].str.split(' to ').str[0]
# Columna fin = final del camino
fin=target[0].str.split(' to ').str[1]
# Columna distancia = distancia entre inicio y fin
distancia=target[1]
print(inicio[0],'->',distancia[0],'->',fin[0])


AlphaCentauri -> 66 -> Snowdin


<a id="6"></a> 

# GENERACIÓN DEL GRAFO

## Genero todos los puntos unicos que existen en el grafo

In [5]:
puntos=np.concatenate((inicio, fin), axis=0)
lista=[puntos[0]]
for i in puntos:
    bandera=True
    for j in lista:
        if(i==j):
            bandera=False
    if(bandera):
        lista.append(i)
puntos=lista
print(puntos)

['AlphaCentauri', 'Snowdin', 'Tambi', 'Faerun', 'Norrath', 'Straylight', 'Tristram', 'Arbre']


## Creo un diccionario de nodos

In [6]:
# Se crea un diccionario id_nodos para manejarlos como indices
# Se crea un diccionario nodos_id que sería la inversa
# Es una idea de Hash- table para acceder a todos los nodos (padre,hijo) de manera mas óptima
id_nodos={}
nodos_id={}
# Matriz de distancias (padre,hijo)=distancia entre ellas
distancias=np.zeros((len(puntos),len(puntos)))
# diccionario de nodos padre e hijos
nodos={}


for i in range(0,len(puntos)):
    id_nodos[puntos[i]]=i
    nodos_id[i]=puntos[i]
    nodos[i]=[]
print(id_nodos)

{'AlphaCentauri': 0, 'Snowdin': 1, 'Tambi': 2, 'Faerun': 3, 'Norrath': 4, 'Straylight': 5, 'Tristram': 6, 'Arbre': 7}


## Asigno a cada nodo sus vecinos y sus distancias

In [7]:

for i in range(0,len(inicio)):
    nodo1=id_nodos[inicio[i]]
    nodo2=id_nodos[fin[i]]
    if(distancias[nodo1,nodo2]==0):
        nodos[nodo1].append(nodo2)
        nodos[nodo2].append(nodo1)
        distancias[nodo1,nodo2]=distancia[i]
        distancias[nodo2,nodo1]=distancia[i]


## Creación de funciones 

In [8]:

def print_path(camino,longitud):
    print('Siendo el más corto camino')
    aux_=''
    for i in range(0,len(camino)):
        aux_=aux_+camino[i]
        if(i!=len(camino)-1):
            aux_=aux_+' -> '
    print(aux_)
    print('distancia=',longitud)
    
def find_mini(aux_j,dis):
    index, value = min(enumerate(dis), key=operator.itemgetter(1))
    return index

def prim_empty():
    longitud=0
    start=0
    visitados=np.zeros(len(puntos))
    visitados[start]=1
    L=[start]
    while(len(L)!=len(puntos)):
        aux_i=[]
        aux_j=[]
        dis=[]
        for padre in L:
            for hijo in nodos[padre]:
                if(visitados[hijo]==0):
                    aux_i.append(padre)
                    aux_j.append(hijo)
                    aux_=distancias[padre,hijo]
                    dis.append(aux_)
        index=find_mini(aux_j,dis)
        min_hijo=aux_j[index]
        longitud=longitud+dis[index]
        L.append(min_hijo)
        visitados[min_hijo]=1
    camino=[]
    for i in L:
        camino.append(nodos_id[i])
    return camino,longitud

def prim(puntos,distancias,nodos,nodos_id):
    longitud=0

    start=0
    visitados=np.zeros(len(puntos))
    visitados[start]=1
    L=[start]
    while(len(L)!=len(puntos)):
        aux_i=[]
        aux_j=[]
        dis=[]
        for padre in L:
            for hijo in nodos[padre]:
                if(visitados[hijo]==0):
                    aux_i.append(padre)
                    aux_j.append(hijo)
                    aux_=distancias[padre,hijo]
                    dis.append(aux_)
        index=find_mini(aux_j,dis)
        min_hijo=aux_j[index]
        longitud=longitud+dis[index]
        L.append(min_hijo)
        visitados[min_hijo]=1
    camino=[]
    for i in L:
        camino.append(nodos_id[i])
    return camino,longitud

def datathon_conda(path_ruta):
    target=pd.read_csv(path_ruta,sep= ' =',header=None)
    # Columna inicio=partida del camino
    inicio=target[0].str.split(' to ').str[0]
    # Columna fin = final del camino
    fin=target[0].str.split(' to ').str[1]
    # Columna distancia = distancia entre inicio y fin
    distancia=target[1]
    #print(inicio[0],'->',distancia[0],'->',fin[0])

    puntos=np.concatenate((inicio, fin), axis=0)
    lista=[puntos[0]]
    for i in puntos:
        bandera=True
        for j in lista:
            if(i==j):
                bandera=False
        if(bandera):
            lista.append(i)
    puntos=lista
    #print(puntos)

    # Se crea un diccionario id_nodos para manejarlos como indices
    # Se crea un diccionario nodos_id que sería la inversa
    # Es una idea de Hash- table para acceder a todos los nodos (padre,hijo) de manera mas óptima
    id_nodos={}
    nodos_id={}
    # Matriz de distancias (padre,hijo)=distancia entre ellas
    distancias=np.zeros((len(puntos),len(puntos)))
    # diccionario de nodos padre e hijos
    nodos={}


    for i in range(0,len(puntos)):
        id_nodos[puntos[i]]=i
        nodos_id[i]=puntos[i]
        nodos[i]=[]
    #print(id_nodos)


    for i in range(0,len(inicio)):
        nodo1=id_nodos[inicio[i]]
        nodo2=id_nodos[fin[i]]
        if(distancias[nodo1,nodo2]==0):
            nodos[nodo1].append(nodo2)
            nodos[nodo2].append(nodo1)
            distancias[nodo1,nodo2]=distancia[i]
            distancias[nodo2,nodo1]=distancia[i]
    path,longitud=prim(puntos,distancias,nodos,nodos_id)
    print_path(path,longitud)
    #return path



In [9]:
path,longitud=prim_empty()

In [10]:
print_path(path,longitud)

Siendo el más corto camino
AlphaCentauri -> Tristram -> Straylight -> Norrath -> Faerun -> Snowdin -> Tambi -> Arbre
distancia= 134.0


<a id="7"></a> 

# PRUEBO EL ALGORITMO

Las funciones datathon_conda

In [12]:
path_ruta='./rutas1.txt'
datathon_conda(path_ruta)

Siendo el más corto camino
London -> Dublin -> Belfast
distancia= 605.0


In [13]:
path_ruta='./rutas.txt'
datathon_conda(path_ruta)

Siendo el más corto camino
AlphaCentauri -> Tristram -> Straylight -> Norrath -> Faerun -> Snowdin -> Tambi -> Arbre
distancia= 134.0


In [14]:
3 + 27 + 9 + 63 + 12 + 22 + 40

176

In [ ]:
## Tristram->AlphaCentauri->Norrath->Straylight->Faerun->SnowdinTambi->Arbre